# make-labels.ipynb

This notebook constructs a YOLOv8 training set. 

INPUT: A **rawdata** folder containing images and pickle files constructed by **detect_rb_damage.ipynb**.

OUTPUT: A training set folder.

## References

https://docs.ultralytics.com/datasets/detect/

https://christianbernecker.medium.com/convert-bounding-boxes-from-coco-to-pascal-voc-to-yolo-and-back-660dc6178742

In [1]:
import glob
import pickle
import pprint
import cv2

In [2]:
##########
# Bounding format conversions frpm https://christianbernecker.medium.com/convert-bounding-boxes-from-coco-to-pascal-voc-to-yolo-and-back-660dc6178742

# Pascal_VOC (Pascal Visual Object Classes)
# [xmin, ymin, xmax, ymax] (in pixels)
# Used by OpenCV

# YOLO
# [xcenter, ycenter, width, height] normalized (0 .. 1)
# Used in YOLO training sets


# Convert Coco bb to Pascal_Voc bb
def coco_to_pascal_voc(x1, y1, w, h):
    return [x1, y1, x1 + w, y1 + h]

# Convert Coco bb to Yolo
def coco_to_yolo(x1, y1, w, h, image_w, image_h):
    return [((2*x1 + w)/(2*image_w)) , ((2*y1 + h)/(2*image_h)), w/image_w, h/image_h]

# Convert Pascal_Voc bb to Coco bb
def pascal_voc_to_coco(x1, y1, x2, y2):
    return [x1,y1, x2 - x1, y2 - y1]

# Convert Pascal_Voc bb to Yolo
def pascal_voc_to_yolo(x1, y1, x2, y2, image_w, image_h):
    return [((x2 + x1)/(2*image_w)), ((y2 + y1)/(2*image_h)), (x2 - x1)/image_w, (y2 - y1)/image_h]


# Convert Yolo bb to Coco bb
def yolo_to_coco(x_center, y_center, w, h, image_w, image_h):
    w = w * image_w
    h = h * image_h
    x1 = ((2 * x_center * image_w) - w)/2
    y1 = ((2 * y_center * image_h) - h)/2
    return [x1, y1, w, h]

# Convert Yolo bb to Pascal_voc bb
def yolo_to_pascal_voc(x_center, y_center, w, h,  image_w, image_h):
    w = w * image_w
    h = h * image_h
    x1 = ((2 * x_center * image_w) - w)/2
    y1 = ((2 * y_center * image_h) - h)/2
    x2 = x1 + w
    y2 = y1 + h
    return [x1, y1, x2, y2]


In [3]:
def proportions_to_pascal_voc(x1, y1, x2, y2, w, h):
    """ Converts bounding box 'proportions' format in pickle file to pascal_voc """
    x1 = round(x1 * w)
    y1 = round(y1 * h)
    x2 = round(x2 * w)
    y2 = round(y2 * h)
    return [x1, y1, x2, y2]

# proportions_to_pascal_voc(0.0, 0.44868666, 0.16984966, 0.99017185, 1920, 1080)
# [0, 485, 326, 1069]

In [4]:
def displaypkl(pklfilepath):
    """ Displays contents of a pickle file """
    pp = pprint.PrettyPrinter(depth=3)
    pklfile = open(pklfilepath, 'rb')
    print()
    print(pklfile)
    mydict = pickle.load(pklfile)
    pp.pprint(mydict)

pklfilepath = '../rawdata/IMG_20221115_111930.pkl'
displaypkl(pklfilepath)


<_io.BufferedReader name='../rawdata/IMG_20221115_111930.pkl'>
{'box_classes': array([2., 3.], dtype=float32),
 'box_detections': 2,
 'box_scores': array([0.80570376, 0.5608622 ], dtype=float32),
 'boxes': array([[0.4425143 , 0.        , 0.9808852 , 0.16115636],
       [0.44868666, 0.        , 0.99017185, 0.16984966]], dtype=float32),
 'cuts': [[255,
           933,
           229,
           931,
           221,
           916,
           206,
           905,
           199,
           893,
           193,
           878,
           193,
           863,
           199,
           848,
           195,
           839,
           199,
           832,
           214,
           829,
           259,
           856,
           266,
           876,
           270,
           921,
           266,
           928,
           255,
           933],
          [333,
           684,
           294,
           683,
           272,
           677,
           264,
           671,
           258,
     

In [5]:
def pkl2label(pklfilepath):
    """
    Uses data contained in a pickle file to build a YOLOv8 label.
    Classes in
    """     
    pp = pprint.PrettyPrinter(depth=3)
    pklfile = open(pklfilepath, 'rb')
    mydict = pickle.load(pklfile)
    img_height = mydict['height']
    img_width = mydict['width']

    s =''
    boxes = mydict.get('boxes')
    if boxes.any():
        for i, box in enumerate(boxes):
            s += f"{str(int(mydict['box_classes'][i] - 1))} "
            x1 = box[1] * img_width
            y1 = box[0] * img_height
            x2 = box[3] * img_width
            y2 = box[2] * img_height
            yolo = pascal_voc_to_yolo(x1, y1, x2, y2, img_width, img_height)
            for n in yolo:
                s += f'{n:.6f} '
            s += '\n'
        
    cuts = mydict.get('cuts')
    if cuts:
        for vcut in cuts:
            # print(cut)
            xlist = vcut[::2]
            ylist = vcut[1::2]
            x1 = min(xlist)
            x2 = max(xlist)
            y1 = min(ylist)
            y2 = max(ylist)
            yolo = pascal_voc_to_yolo(x1=min(xlist), y1=min(ylist), x2=max(xlist), y2=max(ylist), image_w=mydict['width'], image_h=mydict['height'])
            s += '5 '
            for n in yolo:
                s += f'{n:.6f} '
            s += '\n'

    label = s
    return label

label = pkl2label('../rawdata/IMG_20221115_111930.pkl')
print(label)

1 0.080578 0.711700 0.161156 0.538371 
2 0.084925 0.719429 0.169850 0.541485 
5 0.120573 0.815741 0.040104 0.096296 
5 0.157292 0.605556 0.045833 0.055556 



In [8]:
def show_annotated_image(imagepath):
    """

    """
    img = cv2.imread(imagepath)
    img_h, img_w, _ = img.shape

    with open(imagepath.replace('.jpg', '.txt'), 'r') as f:
        annotations = f.read()

    annotations = annotations.rstrip().split('\n')
    for a in annotations:
        a = a.split()
        print(a)
        r = yolo_to_pascal_voc(
            x_center=float(a[1]), y_center=float(a[2]), w=float(a[3]), h=float(a[4]), image_w=img_w, image_h=img_h
            )
        r = [round(float(number)) for number in r]
        print(r)
        cv2.rectangle(img, (r[0], r[1]), (r[2], r[3]), color=(0,0,255), thickness=2)
    cv2.imshow(pklfilepath, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
  
imagepath = '../rawdata/IMG_20221115_111930.jpg'
show_annotated_image(imagepath)

['1', '0.080578', '0.711700', '0.161156', '0.538371']
[0, 478, 309, 1059]
['2', '0.084925', '0.719429', '0.169850', '0.541485']
[0, 485, 326, 1069]
['5', '0.120573', '0.815741', '0.040104', '0.096296']
[193, 829, 270, 933]
['5', '0.157292', '0.605556', '0.045833', '0.055556']
[258, 624, 346, 684]


In [7]:
# MAIN

pklfiles = sorted(glob.glob('../rawdata/*.pkl'))
for pklfilepath in pklfiles:
    print(pklfilepath)
    label = pkl2label(pklfilepath)
    print(label)
    labelfilepath = pklfilepath.replace('.pkl', '.txt')
    with open(labelfilepath, 'w') as f:
        f.write(label)

print('FINISHED')


../rawdata/IMG_20221115_111715.pkl
1 0.412740 0.957619 0.027446 0.064669 

../rawdata/IMG_20221115_111717.pkl
1 0.326654 0.905099 0.031336 0.075804 

../rawdata/IMG_20221115_111718.pkl
1 0.265745 0.901488 0.031685 0.086740 

../rawdata/IMG_20221115_111720.pkl
1 0.307385 0.887163 0.034877 0.092465 

../rawdata/IMG_20221115_111721.pkl
0 0.400608 0.888641 0.040178 0.080610 
1 0.400536 0.888184 0.038516 0.080023 

../rawdata/IMG_20221115_111722.pkl
2 0.533225 0.878152 0.030887 0.083380 

../rawdata/IMG_20221115_111730.pkl
0 0.278370 0.887573 0.079645 0.132582 

../rawdata/IMG_20221115_111735.pkl
1 0.292474 0.882333 0.083002 0.196721 
0 0.293772 0.882690 0.081949 0.194394 

../rawdata/IMG_20221115_111736.pkl
1 0.147517 0.890944 0.107638 0.163501 
0 0.149452 0.887981 0.103969 0.160520 

../rawdata/IMG_20221115_111740.pkl
1 0.058764 0.927580 0.040781 0.084909 

../rawdata/IMG_20221115_111741.pkl
1 0.128185 0.910724 0.044334 0.086248 

../rawdata/IMG_20221115_111743.pkl
1 0.043720 0.888382 0.0